In [53]:
# Crear la conex amb la db MYSQL
import mysql.connector

db = mysql.connector.connect(
    host="localhost",
    port="3306",
    username="root",
    password="123321",
    database="tenfe2"
)

In [54]:
colors = {
    'R1': 'bg-cyan-600',
    'R3': 'bg-red-600',
    'R4': 'bg-orange-600',
    'R11': 'bg-sky-600',
    'R12': 'bg-yellow-300',
}

In [55]:
import pandas as pd

In [56]:
# Llegim el excel i el guardem en un dataframe
df = pd.read_excel('./Horaris.xlsx')

In [57]:
# Creem un cursor per poder fer les consultes
cursor = db.cursor()

In [58]:
df

,LINIA,ID_PARADA,HORA,ID_VIAJE,ORDEN
0,R1,L’Hospitalet de Llobregat,05:48:00,1,1
1,R1,Barcelona Sants,05:55:00,1,2
2,R1,Barcelona Pl.Catalunya,06:00:00,1,3
3,R1,Barcelona Arc de Triomf,06:03:00,1,4
4,R1,Barcelona El Clot Aragó,06:07:00,1,5
...,...,...,...,...,...
10503,R12,Castellnou de Seana,07:56:00,484,5
10504,R12,Golmés,07:59:00,484,6
10505,R12,Mollerussa,08:02:00,484,7
10506,R12,Bell-lloc d’Urgell,08:09:00,484,10


In [59]:
# Agafem tots els valors unics de LINIA
linies = df['LINIA'].unique()
# Les afegim a la db
for linia in linies:
    sql = "SELECT * FROM tenfe2.routes WHERE name = %s"
    cursor.execute(sql, (linia,))
    result = cursor.fetchall()

    if not result:
        sql = "INSERT INTO tenfe2.routes (name,colour) VALUES (%s,%s)"
        cursor.execute(sql, (linia, colors[linia]))
        db.commit()

In [60]:
# Agafem tots els valors unics de ID_PARADA
estacions = df['ID_PARADA'].unique()
# Les afegim a la db
for es in estacions:
    sql = "SELECT * FROM tenfe2.stations WHERE name = %s"
    cursor.execute(sql, (es,))
    result = cursor.fetchall()

    if not result:
        sql = "INSERT INTO tenfe2.stations (name) VALUES (%s)"
        cursor.execute(sql, (es,))
        db.commit()

In [61]:
# Canviem el format de HORA  de 08:00:00 a 08:00
df['HORA'] = df['HORA'].apply(lambda x: x.strftime('%H:%M'))

In [62]:
# Inserim a la db els schedules i route_trains
for row in df.itertuples():
    sql = "SELECT * FROM tenfe2.schedules WHERE train_num = %s AND stop_number = %s"
    cursor.execute(sql, (row.ID_VIAJE, row.ORDEN))
    result = cursor.fetchall()

    if not result:
        sql = "INSERT INTO tenfe2.schedules (station_id, time, train_num, stop_number) VALUES (%s,%s,%s,%s)"

        stId = None
        sql2 = "SELECT id FROM tenfe2.stations WHERE name = %s"
        cursor.execute(sql2, (row.ID_PARADA,))
        result = cursor.fetchall()
        if result:
            stId = result[0][0]
            cursor.execute(sql, (stId, row.HORA, row.ID_VIAJE, row.ORDEN))
            db.commit()

In [63]:
# Creem un df amb tots els valors unics de ID_VIAJE i ORDEN
df2 = df[['LINIA','ID_VIAJE']].drop_duplicates()

df2

,LINIA,ID_VIAJE
0,R1,1
26,R1,2
43,R1,3
60,R1,4
86,R1,5
...,...,...
10388,R12,479
10422,R12,480
10456,R12,481
10490,R12,482


In [64]:
# Inserim a la db els route_trains
for row in df2.itertuples():
    sql = "SELECT * FROM tenfe2.route_trains WHERE route_id = %s AND train_num = %s"
    cursor.execute(sql, (row.LINIA, row.ID_VIAJE))
    result = cursor.fetchall()

    if not result:
        sql = "INSERT INTO tenfe2.route_trains (route_id, train_num) VALUES (%s,%s)"

        cursor.execute(sql, (row.LINIA, row.ID_VIAJE))
        db.commit()
